## Setup

In [1]:
import threading
import time
import queue
import pandas as pd
import numpy as np
from pynq.overlays.base import BaseOverlay
from random import randint
base = BaseOverlay("base.bit")

In [2]:
btns = base.btns_gpio

def blink(t, f, n):
    '''
    Function to blink the LEDs
    Params:
      t: amount of time to blink for (s)
      f: frequency at which to blink the LED (Hz)
      n: index of the LED (0 to 4) - 4 is rgb LED
    '''
    end_time = time.time() + t
    t_on = 1/(2*f)
    led_on = False
    
    if n == 4:
        while time.time() < end_time:
            if not led_on:
                base.rgbleds[4].on(0b010)
                led_on = True
            else:
                base.rgbleds[4].off()
                led_on = False
            time.sleep(t_on)
        base.rgbleds[4].off()
            
    else:
        while time.time() < end_time:
            base.leds[n].toggle()
            time.sleep(t_on)
        base.leds[n].off()
    
def turn_leds_off():
    
    base.rgbleds[4].off()
    for n in range(4):
        base.leds[n].off()

In [3]:
blink(3,30,4)

## Helper threads for button interrupt and prioritization

In [4]:
def button_t():
    global stop
    while not stop:
        if btns.read() != 0:
            stop = True
    print("Button was pressed! Ending program...")
    
def waiter_t():
    global next_to_eat, a, eating
    
    #initialization - first two philosophers
    next_to_eat = nextq.get()
    eating[next_to_eat] = True
    time.sleep(0.1) #give enough time for that thread to pass the block before running this again
    next_to_eat = nextq.get()
    eating[next_to_eat] = True
    time.sleep(0.1) #give enough time for that thread to pass the block before running this again
        
    while not stop:
        last_finished = q.get(True,20)
        print("Last finished: {}".format(last_finished))
        #determine priority (lower values imply hungrier)
        if next_to_eat == 0 or eating[0]:
            if next_to_eat == 1 or eating[1]:
                if next_to_eat == 2 or eating[2]:
                    min_i = 3 #don't let a philosopher eat twice in a row - will cause issues
                else:
                    min_i = 2
            else:
                min_i = 1
        else:
            min_i = 0
        for i in range(5):
            if (i == next_to_eat) or eating[i]: #don't let a philosopher eat twice in a row - will cause issues
                continue
            if a[i] < a[min_i]:
                min_i = i
        print(a[0],a[1],a[2],a[3],a[4])
        print("Highest priority: {}".format(min_i))
        #determine if that philosopher is able to eat next (forks are available now):
        next_chosen = False
        while not next_chosen:
            if ready(min_i) and able_to_eat(min_i):
                #nextq.put(min_i)
                eating[min_i] = True #block eating now so it doesn't mess up future calculations
                next_to_eat = min_i
                next_chosen = True
                print("{} is able to eat next".format(min_i))
            else:
                #if not, pick the one of these two with the highest priority:
                op1 = (last_finished - 1) % 5
                op2 = (last_finished + 1) % 5
                if (a[op1] < a[op2]) and ready(op1) and able_to_eat(op1):
                    eating[op1] = True
                    next_to_eat = op1
                    next_chosen = True
                    print("{} is eating next".format(op1))
                elif ready(op2) and able_to_eat(op2):
                    eating[op2] = True
                    next_to_eat = op2
                    next_chosen = True
                    print("{} is eating next".format(op2))
                elif ready(op1) and able_to_eat(op1):
                    eating[op1] = True
                    next_to_eat = op1
                    next_chosen = True
                    print("{} is eating next".format(op1))
                else:
                    print("Waiting for an eligible philosopher to wake up")
                    time.sleep(0.2)
            if stop:
                break
        time.sleep(0.1) #give enough time for that thread to pass the block before running this again
        

In [5]:
def able_to_eat(index):
    global eating
    return (not eating[index-1]) and (not eating[index]) and (not eating[(index+1)%5])

def ready(index):
    global eating, napping
    return not (eating[index] or napping[index])
            

## Part 2.1

In [6]:
def philosopher_t(left_fork, right_fork, num):
    '''
    Philosopher function to try and acquire resource to eat
    left and right forks - forks (locks)
    num: index representing the LED and thread number.
    '''
    global eating, a, napping, data
    
    using_left_fork = False
    using_right_fork = False
    is_next = False
    
    start_time = int(time.time())
    while not stop:
        start_starving_time = time.time()
        if using_left_fork:
            left_fork.release()
        if using_right_fork:
            right_fork.release()
        
        #Entering Starving state
        data['starve count']['Phil {}'.format(num)] += 1
        print("{}: Philosopher {} is starving".format(time.strftime("%H:%M:%S"),num))
        
        #enter loop to determine who is next to eat
        #want to schedule highest priority that can eat after the next one finishes
        while next_to_eat != num:
            if stop:
                data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                break
            
        
        while not (using_left_fork and using_right_fork):
            using_left_fork = left_fork.acquire(False)
            if using_left_fork:
                using_right_fork = right_fork.acquire(False)
                if not using_right_fork:
                    left_fork.release()
                    using_left_fork = False
                    if stop:
                        data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                        break
                    time.sleep(1)
            else:
                if stop:
                    data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                    break
                time.sleep(1)


        if stop:
            if using_left_fork:
                left_fork.release()
            if using_right_fork:
                right_fork.release()
            using_left_fork = False
            using_right_fork = False
            data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
            break
            
        #has both forks, entering Eating state
        eating[num] = True
        data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
        print("{}: Philosopher {} is eating".format(time.strftime("%H:%M:%S"),num))
        a[num] = a[num] + EAT_TIME
        data['eat count']['Phil {}'.format(num)] += 1
        data['eating time']['Phil {}'.format(num)] += EAT_TIME
        blink(EAT_TIME, 25, num)
        
        #done eating
        left_fork.release()
        right_fork.release()
        napping[num] = True
        eating[num] = False
        q.put(num)
        using_left_fork = False
        using_right_fork = False
        if stop:
            break
        print("{}: Philosopher {} is napping".format(time.strftime("%H:%M:%S"),num))
        data['nap count']['Phil {}'.format(num)] += 1
        data['napping time']['Phil {}'.format(num)] += SLEEP_TIME
        blink(SLEEP_TIME, 0.5, num)
        napping[num] = False
        
    data['time sum']['Phil {}'.format(num)] = data['eating time']['Phil {}'.format(num)] + \
                                              data['napping time']['Phil {}'.format(num)] + \
                                              data['starving time']['Phil {}'.format(num)]
    data['run time']['Phil {}'.format(num)] = int(time.time())-start_time

In [7]:
# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()
rtt = threading.Lock() #right to take

EAT_TIME = 5
SLEEP_TIME = 10

stop = False #stop flag for button interrupt
next_to_eat = 0 #global var for scheduling next to eat

q = queue.Queue() #queue to keep track of the order the philosophers finished eating
a = [0, 0, 0, 0, 0] #array to keep track of priority (how many times the philosophers ate)
eating = [False, False, False, False, False]
napping = [False, False, False, False, False]
nextq = queue.Queue()

cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)

#start with 0 and 2
nextq.put(0)
nextq.put(2)

for i in range(5):
    if i == 0:
        left_fork = fork0
        right_fork = fork4
    elif i == 1:
        left_fork = fork1
        right_fork = fork0
    elif i == 2:
        left_fork = fork2
        right_fork = fork1
    elif i == 3:
        left_fork = fork3
        right_fork = fork2
    elif i == 4:
        left_fork = fork4
        right_fork = fork3
        
    t = threading.Thread(target=philosopher_t, args=(left_fork, right_fork, i))
    threads.append(t)
    t.start()
    
#thread for waiter (prioritization)
t = threading.Thread(target=waiter_t, args=())
threads.append(t)
t.start()
    
#thread for button interrupt
t = threading.Thread(target=button_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))
    
print("Program terminated")
data = data.round(1)
data

08:49:09: Philosopher 0 is starving08:49:09: Philosopher 1 is starving
08:49:09: Philosopher 2 is starving
08:49:09: Philosopher 3 is starving
08:49:09: Philosopher 0 is eating

08:49:09: Philosopher 4 is starving
08:49:09: Philosopher 2 is eating
08:49:14: Philosopher 0 is nappingLast finished: 0
5 0 5 0 0
Highest priority: 1
4 is eating next

08:49:14: Philosopher 4 is eating
08:49:15: Philosopher 2 is napping
Last finished: 2
5 0 5 0 5
Highest priority: 1
1 is able to eat next
08:49:15: Philosopher 1 is eating
08:49:19: Philosopher 4 is nappingLast finished: 4

5 5 5 0 5
Highest priority: 3
3 is able to eat next
08:49:20: Philosopher 3 is eating08:49:20: Philosopher 1 is napping

Last finished: 1
5 5 5 5 5
Highest priority: 0
Waiting for an eligible philosopher to wake up
Waiting for an eligible philosopher to wake up
Waiting for an eligible philosopher to wake up
Waiting for an eligible philosopher to wake up
Waiting for an eligible philosopher to wake up
Waiting for an eligible ph

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,76.0,5.0,5.0,5.0,0.4,25.0,50.0,75.4
Phil 1,82.0,5.0,5.0,5.0,6.1,25.0,50.0,81.1
Phil 2,77.0,5.0,5.0,5.0,0.8,25.0,50.0,75.8
Phil 3,77.0,5.0,4.0,5.0,11.0,25.0,40.0,76.0
Phil 4,82.0,5.0,5.0,5.0,5.7,25.0,50.0,80.7


## Part 2.2

In [8]:
def philosopher_t(left_fork, right_fork, num):
    '''
    Philosopher function to try and acquire resource to eat
    left and right forks - forks (locks)
    num: index representing the LED and thread number.
    '''
    global eating, a, napping, data
    
    using_left_fork = False
    using_right_fork = False
    is_next = False
    has_rtt = False
    
    start_time = int(time.time())
    while not stop:
        start_starving_time = time.time()
        if using_left_fork:
            left_fork.release()
        if using_right_fork:
            right_fork.release()
        
        #Entering Starving state
        data['starve count']['Phil {}'.format(num)] += 1
        print("{}: Philosopher {} is starving".format(time.strftime("%H:%M:%S"),num))
        
        #enter loop to determine who is next to eat
        #want to schedule highest priority that can eat after the next one finishes
        while next_to_eat != num:
            if stop:
                data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                break
            
        
        while not (using_left_fork and using_right_fork):
            using_left_fork = left_fork.acquire(False)
            if using_left_fork:
                using_right_fork = right_fork.acquire(False)
                if not using_right_fork:
                    left_fork.release()
                    using_left_fork = False
                    if stop:
                        data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                        break
                    time.sleep(1)
            else:
                if stop:
                    data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
                    break
                time.sleep(1)


        if stop:
            if using_left_fork:
                left_fork.release()
            if using_right_fork:
                right_fork.release()
            using_left_fork = False
            using_right_fork = False
            data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
            break
            
        #has both forks, entering Eating state
        eating[num] = True
        data['starving time']['Phil {}'.format(num)] += time.time()-start_starving_time
        print("{}: Philosopher {} is eating".format(time.strftime("%H:%M:%S"),num))
        eat_time = randint(5,15)
        a[num] = a[num] + eat_time
        data['eat count']['Phil {}'.format(num)] += 1
        data['eating time']['Phil {}'.format(num)] += eat_time
        blink(eat_time, 25, num)
        
        #done eating
        left_fork.release()
        right_fork.release()
        napping[num] = True
        eating[num] = False
        q.put(num)
        using_left_fork = False
        using_right_fork = False
        if stop:
            break
        print("{}: Philosopher {} is napping".format(time.strftime("%H:%M:%S"),num))
        sleep_time = randint(2,eat_time)
        data['nap count']['Phil {}'.format(num)] += 1
        data['napping time']['Phil {}'.format(num)] += sleep_time
        blink(sleep_time, 0.5, num)
        napping[num] = False
        
    data['time sum']['Phil {}'.format(num)] = data['eating time']['Phil {}'.format(num)] + \
                                              data['napping time']['Phil {}'.format(num)] + \
                                              data['starving time']['Phil {}'.format(num)]
    data['run time']['Phil {}'.format(num)] = int(time.time())-start_time

In [9]:
# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()

stop = False #stop flag for button interrupt
next_to_eat = 0 #global var for scheduling next to eat

q = queue.Queue() #queue to keep track of the order the philosophers finished eating
a = [0, 0, 0, 0, 0] #array to keep track of priority (how long the philosophers ate)
eating = [False, False, False, False, False]
napping = [False, False, False, False, False]
nextq = queue.Queue()

cols = ['run time', 'eat count', 'nap count', 'starve count', 'starving time',
      'eating time', 'napping time', 'time sum']
rows = ['Phil 0', 'Phil 1', 'Phil 2', 'Phil 3', 'Phil 4']
data = pd.DataFrame(np.zeros((5,8)),rows,cols)

#start with 0 and 2
nextq.put(0)
nextq.put(2)

for i in range(5):
    if i == 0:
        left_fork = fork0
        right_fork = fork4
    elif i == 1:
        left_fork = fork1
        right_fork = fork0
    elif i == 2:
        left_fork = fork2
        right_fork = fork1
    elif i == 3:
        left_fork = fork3
        right_fork = fork2
    elif i == 4:
        left_fork = fork4
        right_fork = fork3
        
    t = threading.Thread(target=philosopher_t, args=(left_fork, right_fork, i))
    threads.append(t)
    t.start()
    
#thread for waiter (prioritization)
t = threading.Thread(target=waiter_t, args=())
threads.append(t)
t.start()
    
#thread for button interrupt
t = threading.Thread(target=button_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))
    
print("Program terminated")
data = data.round(1)
data

08:51:20: Philosopher 0 is starving08:51:20: Philosopher 1 is starving
08:51:20: Philosopher 2 is starving

08:51:20: Philosopher 3 is starving
08:51:20: Philosopher 4 is starving08:51:20: Philosopher 0 is eating

08:51:20: Philosopher 2 is eating
08:51:27: Philosopher 2 is nappingLast finished: 2
11 0 7 0 0
Highest priority: 1
3 is eating next
08:51:27: Philosopher 3 is eating

08:51:29: Philosopher 2 is starving
08:51:31: Philosopher 0 is nappingLast finished: 0

11 0 7 7 0
Highest priority: 1
1 is able to eat next
08:51:32: Philosopher 1 is eating
08:51:35: Philosopher 3 is nappingLast finished: 3

11 11 7 7 0
Highest priority: 4
4 is able to eat next
08:51:35: Philosopher 4 is eating
08:51:37: Philosopher 3 is starving
08:51:37: Philosopher 0 is starving
08:51:43: Philosopher 1 is nappingLast finished: 1
11 11 7 7
 15
Highest priority: 2
2 is able to eat next
08:51:43: Philosopher 2 is eating
08:51:45: Philosopher 1 is starving
08:51:50: Philosopher 4 is nappingLast finished: 4

11

,run time,eat count,nap count,starve count,starving time,eating time,napping time,time sum
Phil 0,190.0,6.0,6.0,7.0,94.2,70.0,40.0,204.2
Phil 1,194.0,7.0,7.0,7.0,86.6,71.0,35.0,192.6
Phil 2,201.0,9.0,8.0,9.0,79.6,78.0,42.0,199.6
Phil 3,193.0,7.0,7.0,7.0,79.5,74.0,38.0,191.5
Phil 4,198.0,7.0,6.0,7.0,82.1,87.0,27.0,196.1
